In [262]:
common_pipeline = Pipeline(
    steps=[
        ('imp', imputer),
        ('ohe', ohe_transform),
        ('classifier', LogisticRegression(max_iter=1000))
    ])

In [190]:
imbalanced_nans = ['employment_industry', 'employment_occupation']
imbalanced_nan_num = ['health_insurance']
ord_num_columns = ['h1n1_concern', 'h1n1_knowledge', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'household_adults', 'household_children']
ord_obj_columns = ['age_group', 'education', 'income_poverty']
cat_only_columns = ['race', 'sex', 'marital_status', 'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa', 'employment_industry', 'employment_occupation', 'health_insurance']

all_columns = list(X.columns.values)

In [191]:
first_imputer = PandasSimpleImputer(strategy='constant', fill_value='unknown')
second_imputer = PandasSimpleImputer(strategy='constant', fill_value=2)
third_imputer = PandasSimpleImputer(strategy='most_frequent')

In [192]:
impute_transform_1 = PandasTransformer(transformers=[('unk', first_imputer, imbalanced_nans)])
impute_transform_2 = PandasTransformer(transformers=[('unk2', second_imputer, imbalanced_nan_num)])
impute_transform_3 = ColumnTransformer(transformers=[('imp', third_imputer, all_columns)])

In [162]:
common_pipeline = Pipeline(
    steps=[
        ('imp_unk', impute_transform_1),
        ('imp_unk_2', impute_transform_2),
        ('imp_3', impute_transform_3)
    ])

In [166]:
X_train

In [165]:
common_pipeline.fit_transform(X_train)

In [107]:
linear_preprocessor = ColumnTransformer(
    transformers=[
        ('imputer', common_imputer, )
    ])

In [69]:
X_train.isna().sum().count()

35

In [72]:
X_train_imp = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns, index=X_train.index)

In [73]:
X_train_imp

,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,doctor_recc_h1n1,doctor_recc_seasonal,chronic_med_condition,child_under_6_months,health_worker,health_insurance,opinion_h1n1_vacc_effective,opinion_h1n1_risk,opinion_h1n1_sick_from_vacc,opinion_seas_vacc_effective,opinion_seas_risk,opinion_seas_sick_from_vacc,age_group,education,race,sex,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
respondent_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
18081,2,2,0,1,0,1,0,1,1,0,1,0,0,0,1,3,3,2,3,3,1,35 - 44 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Married,Own,Employed,dqpwygqj,"MSA, Principle City",1,0,fcxhlnwr,xtkaffoo
4201,3,2,0,1,0,1,1,1,1,1,1,1,0,0,1,4,2,4,5,4,5,65+ Years,12 Years,White,Female,"<= $75,000, Above Poverty",Not Married,Own,Not in Labor Force,mlyzmhmf,"MSA, Not Principle City",0,0,fcxhlnwr,xtkaffoo
1237,2,1,0,1,0,1,0,0,1,0,1,1,0,0,1,2,4,1,2,4,2,35 - 44 Years,College Graduate,White,Female,"> $75,000",Married,Own,Employed,fpwskwrf,"MSA, Not Principle City",1,0,wxleyezf,emcorrxb
15265,1,1,0,1,0,1,0,0,1,0,0,0,0,0,1,5,2,2,5,2,2,18 - 34 Years,College Graduate,White,Male,"<= $75,000, Above Poverty",Married,Own,Employed,fpwskwrf,"MSA, Not Principle City",0,0,fcxhlnwr,xtkaffoo
19302,1,1,0,0,0,1,0,0,1,0,0,0,0,0,1,3,4,4,4,4,2,18 - 34 Years,12 Years,White,Female,Below Poverty,Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",1,3,cfqqtusy,mxkfnird
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8746,2,2,0,0,0,1,0,0,0,1,1,0,0,0,1,4,2,1,5,2,1,55 - 64 Years,Some College,White,Female,"<= $75,000, Above Poverty",Married,Own,Employed,fpwskwrf,"MSA, Not Principle City",2,0,fcxhlnwr,mxkfnird
6158,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,5,1,1,1,1,1,35 - 44 Years,< 12 Years,White,Female,Below Poverty,Not Married,Rent,Not in Labor Force,lzgpxyit,Non-MSA,0,0,fcxhlnwr,xtkaffoo
6260,1,1,0,1,0,1,0,0,0,0,1,0,0,0,1,3,2,1,4,2,1,55 - 64 Years,College Graduate,White,Male,"> $75,000",Married,Own,Employed,mlyzmhmf,"MSA, Principle City",1,0,wxleyezf,emcorrxb


In [485]:
common_pipe = Pipeline([('imputer', SimpleImputer(strategy='most_frequent', add_indicator=True))])

### The Linear pipe

In [486]:
all_columns = list(X.columns.values)

In [490]:
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

In [491]:
linear_preprocessor = ColumnTransformer([('common_pipe', common_pipe, all_columns),
                                         ('ohe', categorical_transformer, all_columns)])

In [493]:
#pd.DataFrame(imputer.transform(X_train), columns=X_train.columns, index=X_train.index)

linear_preprocessor.fit_transform(X_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

### The Tree pipe

In [469]:
def ord_encode(df):
    
    """
    This function takes a dataframe and encodes some of the
    categorical variables as ordinal, according to a pre-defined
    list of values. This operation is in place.
    """

    df = pd.DataFrame(df)
    
    age_group_values = ['18 - 34 Years', '35 - 44 Years', '45 - 54 Years', '55 - 64 Years', '65+ Years']
    education_values = ['< 12 Years', '12 Years', 'Some College', 'College Graduate']
    income_poverty_values = ['Below Poverty', '<= $75,000, Above Poverty', '> $75,000']

    values_dict = {'age_group': age_group_values, 'education': education_values, 'income_poverty': income_poverty_values}

    for column_name, values_list in values_dict.items():    
        for num, value in enumerate(values_list):
            df.loc[df[column_name] == value, column_name] = int(num)
        df[column_name] = df[column_name].astype(int)

In [470]:
tree_pipe = Pipeline([('common_pipe', common_pipe),
                      ('ord_encoder', FunctionTransformer(ord_encode))])

In [471]:
tree_pipe.fit_transform(X_train)

KeyError: 'age_group'

In [387]:
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

#imputer.fit(X_train)

#X_train = pd.DataFrame(imputer.transform(X_train), columns=X_train.columns, index=X_train.index)

In [280]:
class PandasSimpleImputer(SimpleImputer):
    """
    A wrapper around 'SimpleImputer' to return data frames with columns.
    """

    def fit(self, X, y=None):
        self.columns = X.columns
        return super().fit(X, y)

    def transform(self, X):
        return pd.DataFrame(super().transform(X), columns=self.columns)


In [266]:
ord_encoder = FunctionTransformer(ord_obj_as_num)
imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

imputer.fit_transform(X_train)

array([[2.0, 2.0, 0.0, ..., 0.0, 'unknown', 'unknown'],
       [3.0, 2.0, 0.0, ..., 0.0, 'unknown', 'unknown'],
       [2.0, 1.0, 0.0, ..., 0.0, 'wxleyezf', 'emcorrxb'],
       ...,
       [1.0, 1.0, 0.0, ..., 0.0, 'wxleyezf', 'emcorrxb'],
       [3.0, 2.0, 0.0, ..., 0.0, 'pxcmvdjn', 'xgwztkwe'],
       [3.0, 1.0, 0.0, ..., 0.0, 'unknown', 'unknown']], dtype=object)

In [267]:
X_train.isna().sum()

h1n1_concern                     70
h1n1_knowledge                   90
behavioral_antiviral_meds        56
behavioral_avoidance            169
behavioral_face_mask             17
behavioral_wash_hands            35
behavioral_large_gatherings      74
behavioral_outside_home          68
behavioral_touch_face           108
doctor_recc_h1n1               1751
doctor_recc_seasonal           1751
chronic_med_condition           782
child_under_6_months            656
health_worker                   643
health_insurance               9790
opinion_h1n1_vacc_effective     323
opinion_h1n1_risk               321
opinion_h1n1_sick_from_vacc     323
opinion_seas_vacc_effective     381
opinion_seas_risk               416
opinion_seas_sick_from_vacc     440
age_group                         0
education                      1133
race                              0
sex                               0
income_poverty                 3556
marital_status                 1143
rent_or_own                 

In [269]:
ord_encoder.fit_transform(X_train)

ValueError: invalid literal for int() with base 10: '35 - 44 Years'